In [2]:
import pandas as pd
import numpy as np
import operator

In [3]:
import os
import dask
import dask.dataframe as dd

In [1]:
from prep import accounts_csvs
# генерация искуственного набора данных:
accounts_csvs(6, 1000000, 500)

ModuleNotFoundError: No module named 'accounts'

1) В account.*.csv найти имя, для которого встречается наибольшее количество положительных значений в стоблце amount 

In [30]:
filename = os.path.join('data', 'accounts.*.csv')
filename

'data\\accounts.*.csv'

In [31]:
df = dd.read_csv(filename)

In [32]:
df.head()

,id,names,amount
0,446,Michael,-416
1,100,Zelda,68
2,303,Michael,171
3,233,Norbert,731
4,352,Laura,815


In [33]:
df.amount.compute()

0          -416
1            68
2           171
3           731
4           815
5           327
6           382
7            91
8           623
9           858
10          101
11          759
12          490
13          -37
14         4468
15          396
16          338
17        10386
18          246
19         2844
20         2994
21         1859
22        -1579
23          789
24         1225
25           78
26          374
27          210
28         2383
29         3126
          ...  
999970      139
999971      536
999972        2
999973    -1080
999974     -874
999975      304
999976     1553
999977      399
999978     2488
999979       79
999980     -255
999981      331
999982       91
999983      882
999984      323
999985     2457
999986     1112
999987      617
999988       -5
999989      165
999990     2279
999991       39
999992     8864
999993      803
999994      357
999995     2415
999996     5231
999997       45
999998      454
999999      440
Name: amount, Length: 60

In [34]:
df.dtypes

id         int64
names     object
amount     int64
dtype: object

In [35]:
df = df[df.amount > 0]

In [36]:
res = df.groupby(df.names).amount.count().idxmax().compute()

In [37]:
res

'Patricia'

2) Эффективно создать csv файлы, содержащие данные из файлов `accounts.*.csv` к которым добавлен столбец, в котором содержится количество сотен из положительного значения amount и 0, если значение в amount отрицательное. 

Пример:

`id,names,amount,new

412,Norbert,1492,14

498,Bob,-91,0

448,Ray,72,0`

In [38]:
filename = os.path.join('data', 'accounts.*.csv')
filename

'data\\accounts.*.csv'

## 1 споcоб

In [39]:
ddf = dd.read_csv(filename)

In [40]:
ddf['new'] = df['amount'] // 100

In [41]:
ddf.head()

,id,names,amount,new
0,446,Michael,-416,NaN
1,100,Zelda,68,0.0
2,303,Michael,171,1.0
3,233,Norbert,731,7.0
4,352,Laura,815,8.0


In [42]:
ddf['new'] = ddf['new'].where(ddf['new'] > 0, 0)

In [43]:
ddf.head()

,id,names,amount,new
0,446,Michael,-416,0.0
1,100,Zelda,68,0.0
2,303,Michael,171,1.0
3,233,Norbert,731,7.0
4,352,Laura,815,8.0


## 2 способ

In [47]:
ddf_2 = dd.read_csv(filename)

In [48]:
ddf_2 = ddf_2.map_partitions(lambda df: df.assign(new = df.amount // 100), 
                  meta={'id': 'i4', 'names': 'object', 'amount': 'i4', 'new': 'i4'})

In [49]:
ddf_2 = ddf_2.map_partitions(lambda df: df.assign(new = df.new.where(cond = df.new > 0, other=0)), 
                  meta={'id': 'i4', 'names': 'object', 'amount': 'i4', 'new': 'i4'})

In [51]:
ddf_2.head()

,id,names,amount,new
0,446,Michael,-416,0
1,100,Zelda,68,0
2,303,Michael,171,1
3,233,Norbert,731,7
4,352,Laura,815,8


3) Эффективно найти все 10 децилей для распределения значения amount. 

In [53]:
import dask.array as da

In [54]:
df = dd.read_csv(filename)

In [55]:
da.percentile(df['amount'].values, q=list(range(10, 101, 10))).compute()

array([  -61.,    41.,   131.,   256.,   468.,   745.,  1260.,  2025.,
        3044., 12552.])

4) Нати 2 последовательно идующие транзакции имеющие наибольшую по модулю разность между значениями amount.

In [58]:
def diff_between_window(df, periods=1):
         before, after = (periods, 0) if periods > 0 else (0, -periods)
         return df.map_overlap(lambda df, periods=1: df.diff(periods),
                           periods, 0, periods=periods)

In [59]:
tr2 = diff_between_window(df['amount'], 1).abs().idxmax()
tr1 = tr2 - 1

In [60]:
dask.compute(tr1, tr2)

(345363, 345364)